In [ ]:
!pip install -Uq openai-agents

In [ ]:
import os 
from dotenv import load_dotenv , find_dotenv # For Loading environment variables
from agents.run import RunConfig

# Load environment variables from .env file
load_dotenv(find_dotenv())

# set SECRET_KEY environment variable
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
model="gemini-2.0-flash"
base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

In [ ]:
from agents import AsyncOpenAI , OpenAIChatCompletionsModel , set_tracing_disabled

set_tracing_disabled(True)

provider = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url=base_url,
)

model = OpenAIChatCompletionsModel(
    openai_client=provider,
    model=model
)

config = RunConfig(
    model=model,
    tracing_disabled=True
)


In [ ]:
from agents import Agent, GuardrailFunctionOutput, RunContextWrapper, Runner, TResponseInputItem, input_guardrail
from pydantic import BaseModel


class SalesManDetectionOutput(BaseModel):
    is_competitor: bool
    is_price: bool

salesman_detection_agent = Agent(
    name="Salesman Detector",
    instructions=("Your Job is to answer user queries about products and offer them to buy But Don't answer questions about competitors and Price or give personal opinions."
                  "If User Ask Question about Competitors or Price or give personal opinions then stop the conversation."
                  "Don't Tell about price of new product."
                  "If user Ask Who is your Competitor then stop the conversation."
                  "If user Ask about Price then stop the conversation."),
    model=model,
    output_type=SalesManDetectionOutput
)

@input_guardrail
async def check_saleman_detection_guardrail(
        ctx : RunContextWrapper[None] , agent : Agent , input : str | list[TResponseInputItem]
    ) -> GuardrailFunctionOutput: # type: ignore

    check_detection = await Runner.run(salesman_detection_agent , input)

    return GuardrailFunctionOutput(
        tripwire_triggered=check_detection.final_output.is_competitor,
        output_info=check_detection.final_output
    )


Salesman_agent = Agent(
    name="Salesman Agent",
    instructions="You are a Salesman Agent. Your job is to answer user queries about products and offer them to buy But Don't answer questions about competitors and Price or give personal opinions.",
    model=model,
    input_guardrails=[check_saleman_detection_guardrail]
)

In [ ]:
from agents import InputGuardrailTripwireTriggered

try:
    result = await Runner.run(Salesman_agent , "What is your Competitor?." , run_config=config)
    print("Guardrail didn't tripwire")
    print("response" , result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("HOmework Guardrail tripped")
    print("Exceptions details" , str(e))